## Yaml

In [1]:
%%writefile file.yml
file_type: csv
dataset_name: testfile
file_name: test_data
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: [City,age,Country]

Overwriting file.yml


In [2]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re


################
# File Reading #
################

def read_config(filepath):
    with open(filepath,'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            loggin.error(exc)
            
def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting testutility.py


In [8]:
# Reading the configuration file
import testutility as util
config_data = util.read_config("file.yml")

In [9]:
config_data

{'file_type': 'csv',
 'dataset_name': 'testfile',
 'file_name': 'test_data',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['City', 'age', 'Country']}

In [10]:
# read the file using config file

file_type = config_data['file_type']
source_file = "./" + config_data['file_name'] + f'.{file_type}'
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()

<IPython.core.display.Javascript object>

C:\Users\RUNA\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3418: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


,city,age,Country
0,Delhi,34,India
1,Lima,30,Peru
2,Istanbul,16,Turkey
3,Riyadh,33,Saudi Arabia


In [11]:
#validate the header of the file
util.col_header_val(df,config_data)

column name and column length validation passed


1

In [12]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['city', 'age', 'country'], dtype='object')
columns of YAML are: ['City', 'age', 'Country']


In [13]:
if util.col_header_val(df,config_data)==0:
    print("validation failed")
    df = pd.read_csv("Empty.csv")
else:
    print("col validation passed")
    # write the code to perform further action
    # in the pipleine

column name and column length validation passed
col validation passed


In [14]:
df.head()

,city,age,country
0,Delhi,34,India
1,Lima,30,Peru
2,Istanbul,16,Turkey
3,Riyadh,33,Saudi Arabia


In [21]:
df.to_csv('test_data_pipe',sep = '|')

In [22]:
import gzip
fp = open("test_data_pipe","rb")
data = fp.read()
bindata = bytearray(data)
with gzip.open("test_data.gzip","wb") as f:
    f.write(bindata)

In [24]:
with gzip.open("test_data.gzip", "rb") as f:
    data = f.read()
data

b'|city|age|country\r\n0|Delhi|34|India\r\n1|Lima|30|Peru\r\n2|Istanbul|16|Turkey\r\n3|Riyadh|33|Saudi Arabia\r\n'

In [25]:
df.describe()

,age
count,4.000000
mean,28.250000
std,8.341663
min,16.000000
25%,26.500000
50%,31.500000
75%,33.250000
max,34.000000


In [26]:
df.shape

(4, 3)